In [1]:
import pandas as pd
from youtubesearchpython import VideosSearch
from youtube_transcript_api._api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
import ollama
import openai
import os
import re
import io
import json
from dotenv import load_dotenv

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set! Please create a .env file.")
openai.api_key = api_key

In [6]:
video_id = 'p43Asp8XCaE'

In [7]:
structured_transcript = None

In [10]:
YouTubeTranscriptApi.list_transcripts(video_id)

In [ ]:
try:
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
    for transcript in transcript_list:
        transcript_type = "Auto-Generated"
        fetchhed = transcript.fetch()
        transcript_text = " ".join(segment.text for segment in fetchhed)
        print(
            f"- Language: {transcript.language}, "
            f"Code: {transcript.language_code}, "
            f"Type: {transcript_type}"
        )
        structured_transcript = fetchhed
        if not transcript.is_generated:
            break
except TranscriptsDisabled:
    print(f"Transcripts are disabled for video ID: {video_id}")
except NoTranscriptFound:
    print(f"No transcripts could be found for video ID: {video_id}")
except Exception as e:
    print(f"An error occurred: {e}")

- Language: English, Code: en, Type: Auto-Generated


In [9]:
transcript_text

'In India, in the late 90s and early 2000s\nwhen someone wanted to buy their first car, they would only go for Maruti 800. If a rich kid has passed some exam, if someone\'s business was running well, or even someone gets\nan award for getting married. Everyone used to buy Maruti\n800 as their first car. Forget about normal people,\neven people like Sachin Tendulkar, Shah Rukh Khan and Boman Irani they had a Maruti 800 as their first car. And the reason was that the 800 was cheap,\nand was affordable for people of that economy, and the markets had no special options. But today, in 2025, everything has changed. Today, we have so many options\nwhich can make us dizzy. And thanks to the IT boom in India,\nbecause the beginners are making in lakhs. And high-earning individuals,\nwith their first-year income, they would go for offroaders like\nThar Roxx or sedans like Virtus. But, I think, though you have\nmoney earlier than the usual you should not go for these cars\nwhen you are a first-ti

In [19]:
# Define the two prompts in our chain
prompt_1_template = """You are an expert data extractor. Your task is to read the following transcript, which could be in any language, and produce a summary **in English**.

The summary must follow these specific rules:
1. Identify every unique vehicle mentioned in the transcript.
2. For each vehicle, create a primary bullet point using its name (e.g., "* 2024 Hyundai Creta").
3. Under each primary bullet point, create a nested list of all its specifications.
4. Each specification in the nested list must use the format "Attribute: Value" (e.g., "- Price: ₹7.5 lakh", "- Color: Blue:").
5. Add a final bullet point for "original_quote": A SHORT, unique, and EXACT quote from the original transcript that clearly identifies this specific car. **This quote MUST be in the original language of the transcript. and MUST be continuous**

---
Transcript to process:
{transcript}
"""

# We explicitly ask for a markdown table and "NA" for missing values.
# Adding "and nothing else" helps prevent the model from adding conversational text.
prompt_2_template = """Structure all of the information given in following summary in a tabular format.
- Create columns for the following attributes:
OEM | Model | Variant | Price | Colour | Odo | Year | Service record | Frame type | Transmission type | Fuel type | Number of owners | RTO  | City | Engine details | Transmission details | Feature details | Rating
- If a piece of data is not available for a specific car, just put NA.
- The output should be a clean markdown table and nothing else.
- Add a final column "original_quote" mentioned in the summary
---
Summary:
{summary}
"""

prompt_3_template = """You are an expert contact information extractor. From the provided text, which includes a video description and parts of a transcript, extract the following details for the car dealer.

The details to extract are:
- "dealer_name": The name of the dealership or person selling the cars.
- "dealer_contact": A phone or mobile number. Extract only the number.
- "dealer_email": An email address.
- "dealer_website": A website URL.
- "dealer_location": The physical address or area mentioned.

Structure your output as a single JSON object. If a piece of information is not found, use `null` for that field's value.

Here is the text to analyze:
---
{contact_corpus}
---
"""

In [33]:
def run_prompt_chain_openai(transcript):
    """
    Executes the two-step prompt chain to extract and structure car information.
    """
    print("--- Step 1: Kicking off the first prompt (Summarization) ---")
    
    # Format the first prompt with the transcript
    prompt_1 = prompt_1_template.format(transcript=transcript)
    
    # Call the LLM for the first time
    response_1 = openai.chat.completions.create(
        model="o4-mini", # Using a more advanced model can yield better results
        messages=[
            {"role": "system", "content": "You are an expert assistant skilled at summarizing technical specifications from text."},
            {"role": "user", "content": prompt_1}
        ]
    )
    summary = response_1.choices[0].message.content
    
    print("\n✅ Intermediate Summary from Prompt 1:\n")
    print(summary)
    
    print("\n--- Step 2: Kicking off the second prompt (Structuring) ---")
    
    # Format the second prompt with the summary from the first call
    prompt_2 = prompt_2_template.format(summary=summary)

    # Call the LLM for the second time
    response_2 = openai.chat.completions.create(
        model="o4-mini",
        messages=[
            {"role": "system", "content": "You are a data formatting assistant. Your job is to convert text into clean markdown tables."},
            {"role": "user", "content": prompt_2}
        ]
    )
    structured_data_string = response_2.choices[0].message.content
    
    print("\n✅ Structured Markdown Table from Prompt 2:\n")
    print(structured_data_string)
    
    return structured_data_string

In [20]:
def run_prompt_chain_ollama(transcript):
    """
    Executes the two-step prompt chain to extract and structure car information.
    """
    print("--- Step 1: Kicking off the first prompt (Summarization) ---")
    
    # Format the first prompt with the transcript
    prompt_1 = prompt_1_template.format(transcript=transcript)
    
    # Call the LLM for the first time
    response_1 = ollama.chat(
        model="qwen3:30b", # Using a more advanced model can yield better results
        messages=[
            {"role": "system", "content": "You are an expert assistant skilled at summarizing technical specifications from text."},
            {"role": "user", "content": prompt_1}
        ], 
        think=True,
        # options={       # Optional: Adjust generation parameters if needed
        #     'temperature': 0.2,
        #     # 'num_predict': 1024 # Limit output length if necessary
        # }
    )
    summary = response_1['message']['content'].strip()
    
    print("\n✅ Intermediate Summary from Prompt 1:\n")
    print(summary)
    
    print("\n--- Step 2: Kicking off the second prompt (Structuring) ---")
    
    # Format the second prompt with the summary from the first call
    prompt_2 = prompt_2_template.format(summary=summary)

    # Call the LLM for the second time
    response_2 = ollama.chat(
        model="qwen3:30b",
        messages=[
            {"role": "system", "content": "You are a data formatting assistant. Your job is to convert text into clean markdown tables"},
            {"role": "user", "content": prompt_2}
        ], 
        think=True
        # options={       # Optional: Adjust generation parameters if needed
        #     'temperature': 0.2,
        #     # 'num_predict': 1024 # Limit output length if necessary
        # }
    )
    structured_data_string = response_2['message']['content'].strip()
    
    print("\n✅ Structured Markdown Table from Prompt 2:\n")
    print(structured_data_string)
    
    return structured_data_string

In [21]:
def parse_markdown_to_dataframe(llm_output_string: str) -> pd.DataFrame:
    """
    Parses a markdown table from a potentially messy LLM output string
    into a clean pandas DataFrame. It handles surrounding text and code fences.
    """
    print("\n--- Step 3: Parsing LLM output into a Pandas DataFrame ---")

    # Use regex to find the markdown table, including the optional ```markdown fence
    # re.DOTALL allows '.' to match newlines
    match = re.search(r"```(?:markdown)?\n([\s\S]*?)\n```", llm_output_string, re.DOTALL)
    
    if match:
        # If a fenced code block is found, use its content
        markdown_table = match.group(1).strip()
    else:
        # If no code block is found, fall back to finding contiguous lines
        # that look like a table.
        lines = llm_output_string.strip().split('\n')
        table_lines = [line for line in lines if line.strip().startswith('|') and line.strip().endswith('|')]
        if not table_lines:
             raise ValueError("Could not find a markdown table in the LLM output.")
        markdown_table = "\n".join(table_lines)
        
    print("\n✅ Successfully Extracted Markdown Table:\n")
    print(markdown_table)
    
    # Use io.StringIO to treat the string as a file for pandas
    # The markdown table uses '|' as a separator.
    # We skip the separator line (e.g., |---|---|) by filtering it out.
    lines = markdown_table.strip().split('\n')
    cleaned_lines = [line for line in lines if not set(line.strip()).issubset({'|', '-', ' ', ':'})]
    
    data_string = "\n".join(cleaned_lines)
    data = io.StringIO(data_string)
    
    # Read the data using the '|' separator
    df = pd.read_csv(data, sep='|', index_col=False)

    # Drop the empty columns created by the leading and trailing '|'
    df = df.drop(df.columns[[0, -1]], axis=1)

    # Strip whitespace from column names and all string cells
    df.columns = df.columns.str.strip()
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

    print("\n✅ Final, Cleaned Pandas DataFrame:\n")
    return df

In [22]:
markdown_output = run_prompt_chain_ollama(transcript_text)

--- Step 1: Kicking off the first prompt (Summarization) ---

✅ Intermediate Summary from Prompt 1:

It seems like you've shared a detailed sales pitch or video script in Hindi, likely for a car dealership or vehicle sales. However, your request isn't explicitly clear. Could you clarify what you need help with? For example:  
- A **translation** of the text into English?  
- A **summary** of the key points?  
- Assistance with **analyzing the content** (e.g., identifying car models, pricing, financing options)?  
- Or something else?  

Let me know, and I’ll tailor my response to your needs! 😊

--- Step 2: Kicking off the second prompt (Structuring) ---

✅ Structured Markdown Table from Prompt 2:

| OEM | Model | Variant | Price | Colour | Odo | Year | Service record | Frame type | Transmission type | Fuel type | Number of owners | RTO  | City | Engine details | Transmission details | Feature details | Rating | original_quote |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|-

In [37]:
markdown_output = run_prompt_chain_openai(transcript_text)

--- Step 1: Kicking off the first prompt (Summarization) ---

✅ Intermediate Summary from Prompt 1:

* 2021 Tata Safari XZ A Plus  
  - Model Year: 2021  
  - Variant: XZ A Plus  
  - Seating Capacity: 7  
  - Sunroof: Panoramic  
  - Transmission: Automatic  
  - Emission Standard: BS6  
  - Price: ₹1,99,000 (negotiable)  
  - Registration: Delhi  
  - Ownership: First owner  
  original_quote: "2021 मॉडल XZ A प्लस टॉप वेरिएंट के अंदर पेनारोमिक सनरूफ के साथ में।"

* 2023 Tata Punch CNG  
  - Fuel Type: CNG (factory-fitted)  
  - Units Available: 3  
  - Mileage: ~5,700–7,000 km  
  - Registration: Delhi  
  - Ownership: First owner  
  - Price: ₹8.5 lakh (negotiable)  
  original_quote: "पंच में हमारे पास टोटल तीन कार हैं। सारे ही लगभग 5 7000 के आसपास चली हुई है।"

* 2023 Tata Nexon Petrol Fairless Plus  
  - Fuel Type: Petrol  
  - Variant: Fairless Plus (sunroof)  
  - Transmission: Automatic  
  - Model Year: 2023  
  - Price: ₹15 lakh  
  original_quote: "2023 मॉडल इसके अंदर आपको 

In [24]:
markdown_output_temp = """| OEM       | Model           | Variant       | Price                       | Colour                   | Odo              | Year | Service record | Frame type | Transmission type | Fuel type | Number of owners | RTO    | City   | Engine details | Transmission details | Feature details                                              | Rating | original_quote                                                                                                                                           |
|-----------|-----------------|---------------|-----------------------------|--------------------------|------------------|------|----------------|------------|-------------------|-----------|------------------|--------|--------|----------------|----------------------|----------------------------------------------------------------|--------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| Tata      | Safari          | XZ A Plus     | ₹1,99,000 (negotiable)      | NA                       | NA               | 2021 | NA             | NA         | Automatic         | NA        | 1                | Delhi  | Delhi  | NA             | NA                   | Panoramic sunroof; Seating capacity 7                         | NA     | "2021 मॉडल XZ A प्लस टॉप वेरिएंट के अंदर पेनारोमिक सनरूफ के साथ में।"                                                                                     |
| Tata      | Punch           | CNG           | ₹8.5 lakh (negotiable)      | NA                       | 5,700–7,000 km   | 2023 | NA             | NA         | NA                | CNG       | 1                | Delhi  | Delhi  | NA             | NA                   | 3 units available                                             | NA     | "पंच में हमारे पास टोटल तीन कार हैं। सारे ही लगभग 5 7000 के आसपास चली हुई है।"                                                                         |
| Tata      | Nexon           | Fairless Plus | ₹15 lakh                    | NA                       | NA               | 2023 | NA             | NA         | Automatic         | Petrol    | NA               | NA     | NA     | NA             | NA                   | Sunroof                                                       | NA     | "2023 मॉडल इसके अंदर आपको ऑटो चाहिए भाई न्यू शेप की Tata Nexon पे 6.5 लाख बचा दिए हैं"                                                                        |
| Tata      | Nexon EV        | Black Beauty  | ₹14.5 lakh                  | Black                    | 2,000 km         | 2024 | NA             | NA         | NA                | Electric  | NA               | NA     | NA     | NA             | NA                   | Long-range; Fast charger (7.2 kW, free); Company warranty    | NA     | "ब्लैक ब्यूटी ईवी नेक्स्ट की न्यू शेप की ये डार्क एडिशन के साथ में है और गाड़ी चली है सिर्फ 2000 किलोमीटर"                                                  |
| Tata      | Nexon EV        | LX Plus       | ₹25 lakh                    | NA                       | 25,000 km        | 2022 | NA             | NA         | NA                | Electric  | NA               | NA     | NA     | NA             | NA                   | Wireless charger; No fast charger                              | NA     | "ये 2022 का मॉडल है ये अच्छा 22 का है ठीक है ये भी लॉन्ग रेंज के साथ बट ये विदाउट फास्ट चार्जिंग के साथ में है।"                                              |
| Tata      | Nexon EV        | Prime         | ₹8,85,000                   | NA                       | 60,000 km        | 2020 | NA             | NA         | NA                | Electric  | NA               | NA     | NA     | NA             | NA                   | NA                                                             | NA     | "2020 मॉडल Nexon EV Prime मिल जाएगा। 60,000 कि.मी. गाड़ी चली है।"                                                                                           |
| Maruti    | Altroz          | Top           | ₹6,99,000 (negotiable)      | Dual Tone                | NA               | NA   | NA             | NA         | Manual            | NA        | NA               | NA     | NA     | NA             | NA                   | NA                                                             | NA     | "Altros आ चुकी है मैनुअल ट्रांसमिशन में टॉप वेरिएंट है। ड्यूल टोन के अंदर गाड़ी है।"                                                                        |
| Maruti    | Altroz          | Top           | ₹7,45,000 (negotiable)      | Dual Tone                | NA               | NA   | NA             | NA         | Automatic         | NA        | NA               | NA     | NA     | NA             | NA                   | NA                                                             | NA     | "अगर ऑटोमैटिक चाहिए तो ऑटोमैटिक में आपको ये मिल जाएगी 7,45,000 की रेंज में"                                                                             |
| Maruti    | Altroz          | Top           | ₹6,40,000 (negotiable)      | White; Red beauty option | 5,200 km         | 2023 | NA             | NA         | Automatic         | NA        | NA               | NA     | NA     | NA             | NA                   | NA                                                             | NA     | "5200 कि.मी. गाड़ी चली है। ... दे देंगे सिर्फ ₹6400 की रेंज में।"                                                                                             |
| Hyundai   | i10             | Sports        | ₹4.5 lakh                   | NA                       | 35,000 km        | 2019 | NA             | NA         | Manual            | Petrol    | NA               | NA     | NA     | NA             | NA                   | Alloy wheels                                                  | NA     | "i10 सर ये हमारे पास 2019 मॉडल आ चुकी है। ₹45 लाख की रेंज में स्पोर्ट्स वेरिएंट आपको देंगे अलॉय व्हील वाले।"                                                 |
| Maruti    | Baleno          | Delta         | ₹5,15,000                   | NA                       | 65,000 km        | 2020 | NA             | NA         | Manual            | Petrol    | NA               | NA     | NA     | NA             | NA                   | NA                                                             | NA     | "Baleno सर कम बजट में लेके आए हैं ₹5,15,000 की रेंज में। Baleno देंगे डेल्टा वेरिएंट। मैनुअल ट्रांसमिशन में गाड़ी रहेगी 2020 मॉडल।"                             |
| Hyundai   | i20             | Sports        | ₹9 lakh                     | NA                       | 12,000 km        | 2024 | NA             | NA         | Automatic         | Petrol    | NA               | NA     | NA     | NA             | NA                   | Company warranty                                              | NA     | "गाड़ी दूंगा मैं आपको ₹9 लाख की रेंज में। स्पोर्ट्स वेरिएंट है। 2024 मॉडल है। 12,000 चली है।"                                                               |
| Maruti    | Celerio         | Top           | ₹5,75,000                   | NA                       | 40,000 km        | 2023 | Yes            | NA         | CVT               | Petrol    | 1                | NA     | NA     | NA             | NA                   | Service record; First owner                                  | NA     | "2023 मॉडल की कंपनी फिटेड सेरियो न्यू शेप के अंदर गाड़ी लेके आ चुके हैं। 40000 विद सर्विस रिकॉर्ड फर्स्ट ओनर..."                                                |
| Honda     | Civic           | ZX            | ₹13,99,000                  | NA                       | 500 km           | 2019 | NA             | NA         | CVT               | Petrol    | NA               | NA     | NA     | NA             | NA                   | Sunroof                                                       | NA     | "2019 मॉडल है। ZX सीवीटी सनरूफ के साथ में है। ...500 कि.मी. गाड़ी चली है। ...13 लाख और 99,000 की रेंज में।"                                                     |
| Renault   | Triber          | RXL           | ₹5 lakh                     | NA                       | NA               | 2022 | NA             | NA         | Manual            | NA        | NA               | NA     | NA     | NA             | NA                   | Seating capacity 7                                           | NA     | "Triber में सेवन सीटर 2022 मॉडल सिर्फ ₹.5 लाख की रेंज में। ये वाली RXL वेरिएंट है।"                                                                     |
| MG        | ZS EV           | NA            | ₹13,99,000                  | NA                       | 300 km           | 2023 | NA             | NA         | NA                | Electric  | 1                | NA     | NA     | NA             | NA                   | Panoramic sunroof                                             | NA     | "MG ZS EV आपके लिए लेके आ चुके है। 2023 मॉडल। चली कितनी है? गाड़ी चली है सिर्फ 300 कि.मी... पैनारोमिक सनरूफ मिल जाती है।"                                  |
| Mahindra  | Thar            | 4x4           | ₹17,50,000                  | NA                       | 30,000 km        | 2023 | NA             | NA         | Automatic         | Petrol    | NA               | NA     | NA     | NA             | NA                   | 3-door; Drive 4×4                                             | NA     | "₹1,75,000 में आपको 4/4 ब्लैक ब्यूटी की Thar मिल रही है। थ्री डोर अनरजिस्टर्ड गाड़ी है।"                                                              |
| Mahindra  | Thar            | 4x2           | ₹15,00,000                  | NA                       | NA               | NA   | NA             | NA         | Automatic         | Petrol    | NA               | NA     | NA     | NA             | NA                   | NA                                                             | NA     | "4 बाय टू चाहिए ऑटो पेट्रोल तो वो भी ऑप्शंस अवेलेबल है। ₹15 लाख की रेंज में"                                                                             |"""

In [38]:
df = parse_markdown_to_dataframe(markdown_output)
df


--- Step 3: Parsing LLM output into a Pandas DataFrame ---

✅ Successfully Extracted Markdown Table:

| OEM       | Model           | Variant       | Price                       | Colour                   | Odo              | Year | Service record | Frame type | Transmission type | Fuel type | Number of owners | RTO    | City   | Engine details | Transmission details | Feature details                                              | Rating | original_quote                                                                                                                                           |
|-----------|-----------------|---------------|-----------------------------|--------------------------|------------------|------|----------------|------------|-------------------|-----------|------------------|--------|--------|----------------|----------------------|----------------------------------------------------------------|--------|---------------------------------------------------------

,OEM,Model,Variant,Price,Colour,Odo,Year,Service record,Frame type,Transmission type,Fuel type,Number of owners,RTO,City,Engine details,Transmission details,Feature details,Rating,original_quote
0,Tata,Safari,XZ A Plus,"₹1,99,000 (negotiable)",NA,NA,2021,NA,NA,Automatic,NA,1,Delhi,Delhi,NA,NA,Panoramic sunroof; Seating capacity 7,NA,"""2021 मॉडल XZ A प्लस टॉप वेरिएंट के अंदर पेनार..."
1,Tata,Punch,CNG,₹8.5 lakh (negotiable),NA,"5,700–7,000 km",2023,NA,NA,NA,CNG,1,Delhi,Delhi,NA,NA,3 units available,NA,"""पंच में हमारे पास टोटल तीन कार हैं। सारे ही ल..."
2,Tata,Nexon,Fairless Plus,₹15 lakh,NA,NA,2023,NA,NA,Automatic,Petrol,NA,NA,NA,NA,NA,Sunroof,NA,"""2023 मॉडल इसके अंदर आपको ऑटो चाहिए भाई न्यू श..."
3,Tata,Nexon EV,Black Beauty,₹14.5 lakh,Black,"2,000 km",2024,NA,NA,NA,Electric,NA,NA,NA,NA,NA,"Long-range; Fast charger (7.2 kW, free); Compa...",NA,"""ब्लैक ब्यूटी ईवी नेक्स्ट की न्यू शेप की ये डा..."
4,Tata,Nexon EV,LX Plus,₹25 lakh,NA,"25,000 km",2022,NA,NA,NA,Electric,NA,NA,NA,NA,NA,Wireless charger; No fast charger,NA,"""ये 2022 का मॉडल है ये अच्छा 22 का है ठीक है य..."
5,Tata,Nexon EV,Prime,"₹8,85,000",NA,"60,000 km",2020,NA,NA,NA,Electric,NA,NA,NA,NA,NA,NA,NA,"""2020 मॉडल Nexon EV Prime मिल जाएगा। 60,000 कि..."
6,Maruti,Altroz,Top,"₹6,99,000 (negotiable)",Dual Tone,NA,NA,NA,NA,Manual,NA,NA,NA,NA,NA,NA,NA,NA,"""Altros आ चुकी है मैनुअल ट्रांसमिशन में टॉप वे..."
7,Maruti,Altroz,Top,"₹7,45,000 (negotiable)",Dual Tone,NA,NA,NA,NA,Automatic,NA,NA,NA,NA,NA,NA,NA,NA,"""अगर ऑटोमैटिक चाहिए तो ऑटोमैटिक में आपको ये मि..."
8,Maruti,Altroz,Top,"₹6,40,000 (negotiable)",White; Red beauty option,"5,200 km",2023,NA,NA,Automatic,NA,NA,NA,NA,NA,NA,NA,NA,"""5200 कि.मी. गाड़ी चली है। ... दे देंगे सिर्फ ..."
9,Hyundai,i10,Sports,₹4.5 lakh,NA,"35,000 km",2019,NA,NA,Manual,Petrol,NA,NA,NA,NA,NA,Alloy wheels,NA,"""i10 सर ये हमारे पास 2019 मॉडल आ चुकी है। ₹45 ..."


In [25]:
df_temp = parse_markdown_to_dataframe(markdown_output_temp)


--- Step 3: Parsing LLM output into a Pandas DataFrame ---

✅ Successfully Extracted Markdown Table:

| OEM       | Model           | Variant       | Price                       | Colour                   | Odo              | Year | Service record | Frame type | Transmission type | Fuel type | Number of owners | RTO    | City   | Engine details | Transmission details | Feature details                                              | Rating | original_quote                                                                                                                                           |
|-----------|-----------------|---------------|-----------------------------|--------------------------|------------------|------|----------------|------------|-------------------|-----------|------------------|--------|--------|----------------|----------------------|----------------------------------------------------------------|--------|---------------------------------------------------------

In [28]:
df = df_temp

In [26]:
from thefuzz import fuzz

def find_best_match_in_transcript(quote, structured_transcript, window_size=3, score_cutoff=85):
    """
    Finds the best match for a quote in a transcript, considering that the quote
    might span multiple chunks and might not be an exact match.

    Args:
        quote (str): The string to search for (from the LLM).
        structured_transcript (list): The list of {'text', 'start', ...} dicts.
        window_size (int): The number of consecutive chunks to combine for searching.
        score_cutoff (int): The minimum fuzzy match score (0-100) to consider a match valid.

    Returns:
        The starting chunk dictionary of the best match if the score is above cutoff, otherwise None.
    """
    if not quote or not structured_transcript:
        return None

    best_match_score = -1
    best_match_chunk = None

    chunks = structured_transcript
    
    # Iterate through each chunk as a potential starting point for a window
    for i in range(len(chunks)):
        # Create a sliding window of text
        window_chunks = chunks[i : i + window_size]
        combined_text = " ".join([c.text for c in window_chunks])

        # Use fuzzy matching to get a similarity score.
        # fuzz.partial_ratio is often good for finding a sub-string within a larger text block.
        # You can also experiment with fuzz.token_set_ratio.
        current_score = fuzz.partial_ratio(quote, combined_text)
        
        # If this window gives a better score, update our best match
        if current_score > best_match_score:
            best_match_score = current_score
            # The relevant chunk is the one at the start of the window
            best_match_chunk = chunks[i]

            # Optimization: If we find a perfect match, we can stop early.
            if best_match_score > 95:
                break
    
    # After checking all windows, decide if the best match is good enough
    if best_match_chunk is not None and best_match_score >= score_cutoff:
        #print(f"Found match for '{quote}' with score {best_match_score} starting at {best_match_chunk.start}s.")
        return best_match_chunk
    else:
        #print(f"No suitable match found for '{quote}'. Best score was {best_match_score}, which is below the cutoff of {score_cutoff}.")
        return None


In [27]:
def find_original_quote_in_transcript(search_quote, structured_transcript):
    """
    For each row in the DataFrame, find the original quote in the transcript
    and add its details in the dataframe.
    """
    start_chunk = find_best_match_in_transcript(
        quote=search_quote, 
        structured_transcript=structured_transcript,
        window_size=3,      # Look at 3 chunks at a time
        score_cutoff=80     # Require at least 85% similarity
    )
    if start_chunk:
        # If we found a match, return the start time and text
        return {
            "start_timestamp": start_chunk.start,
            "video_link": f"https://www.youtube.com/watch?v={video_id}&t={int(start_chunk.start)}s",
        }
    else:
        # If no match was found, return None
        return {
            "start_timestamp": None,
            "video_link": None,
        }

In [29]:
df[['start_timestamp', 'video_link']] = df.apply(lambda x: find_original_quote_in_transcript(x['original_quote'], structured_transcript), axis=1, result_type="expand")
df

,OEM,Model,Variant,Price,Colour,Odo,Year,Service record,Frame type,Transmission type,...,Number of owners,RTO,City,Engine details,Transmission details,Feature details,Rating,original_quote,start_timestamp,video_link
0,Tata,Safari,XZ A Plus,"₹1,99,000 (negotiable)",NA,NA,2021,NA,NA,Automatic,...,1,Delhi,Delhi,NA,NA,Panoramic sunroof; Seating capacity 7,NA,"""2021 मॉडल XZ A प्लस टॉप वेरिएंट के अंदर पेनार...",140.720,https://www.youtube.com/watch?v=lbBDe4UJIgw&t=...
1,Tata,Punch,CNG,₹8.5 lakh (negotiable),NA,"5,700–7,000 km",2023,NA,NA,NA,...,1,Delhi,Delhi,NA,NA,3 units available,NA,"""पंच में हमारे पास टोटल तीन कार हैं। सारे ही ल...",9.280,https://www.youtube.com/watch?v=lbBDe4UJIgw&t=9s
2,Tata,Nexon,Fairless Plus,₹15 lakh,NA,NA,2023,NA,NA,Automatic,...,NA,NA,NA,NA,NA,Sunroof,NA,"""2023 मॉडल इसके अंदर आपको ऑटो चाहिए भाई न्यू श...",243.040,https://www.youtube.com/watch?v=lbBDe4UJIgw&t=...
3,Tata,Nexon EV,Black Beauty,₹14.5 lakh,Black,"2,000 km",2024,NA,NA,NA,...,NA,NA,NA,NA,NA,"Long-range; Fast charger (7.2 kW, free); Compa...",NA,"""ब्लैक ब्यूटी ईवी नेक्स्ट की न्यू शेप की ये डा...",258.239,https://www.youtube.com/watch?v=lbBDe4UJIgw&t=...
4,Tata,Nexon EV,LX Plus,₹25 lakh,NA,"25,000 km",2022,NA,NA,NA,...,NA,NA,NA,NA,NA,Wireless charger; No fast charger,NA,"""ये 2022 का मॉडल है ये अच्छा 22 का है ठीक है य...",NaN,None
5,Tata,Nexon EV,Prime,"₹8,85,000",NA,"60,000 km",2020,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,"""2020 मॉडल Nexon EV Prime मिल जाएगा। 60,000 कि...",324.199,https://www.youtube.com/watch?v=lbBDe4UJIgw&t=...
6,Maruti,Altroz,Top,"₹6,99,000 (negotiable)",Dual Tone,NA,NA,NA,NA,Manual,...,NA,NA,NA,NA,NA,NA,NA,"""Altros आ चुकी है मैनुअल ट्रांसमिशन में टॉप वे...",334.560,https://www.youtube.com/watch?v=lbBDe4UJIgw&t=...
7,Maruti,Altroz,Top,"₹7,45,000 (negotiable)",Dual Tone,NA,NA,NA,NA,Automatic,...,NA,NA,NA,NA,NA,NA,NA,"""अगर ऑटोमैटिक चाहिए तो ऑटोमैटिक में आपको ये मि...",342.960,https://www.youtube.com/watch?v=lbBDe4UJIgw&t=...
8,Maruti,Altroz,Top,"₹6,40,000 (negotiable)",White; Red beauty option,"5,200 km",2023,NA,NA,Automatic,...,NA,NA,NA,NA,NA,NA,NA,"""5200 कि.मी. गाड़ी चली है। ... दे देंगे सिर्फ ...",381.039,https://www.youtube.com/watch?v=lbBDe4UJIgw&t=...
9,Hyundai,i10,Sports,₹4.5 lakh,NA,"35,000 km",2019,NA,NA,Manual,...,NA,NA,NA,NA,NA,Alloy wheels,NA,"""i10 सर ये हमारे पास 2019 मॉडल आ चुकी है। ₹45 ...",412.080,https://www.youtube.com/watch?v=lbBDe4UJIgw&t=...


In [31]:
from datetime import datetime, timezone, timedelta

In [32]:
df['video_id'] = 'lbBDe4UJIgw'
df['video_title'] = 'Some Title'
df['published_at'] = "2023-10-01 12:00:00"  # Placeholder date, replace with actual video publish date
df['channel_title'] = "Some channel"
df['timestamp'] = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

In [33]:
def create_contact_corpus(structured_transcript, description, num_chunks=25):
    """
    Creates a small, targeted text corpus for extracting contact info.

    Args:
        structured_transcript (list): The list of {'text', 'start'} dicts.
        description (str): The full text of the video description.
        num_chunks (int): The number of chunks to take from the start and end.

    Returns:
        str: A combined string of the most relevant text.
    """
    # Get the first N chunks of the transcript
    intro_chunks = structured_transcript[:num_chunks]
    intro_text = " ".join([chunk.text for chunk in intro_chunks])

    # Get the last N chunks of the transcript
    outro_chunks = structured_transcript[-num_chunks:]
    outro_text = " ".join([chunk.text for chunk in outro_chunks])

    # Combine all relevant text into one block
    corpus = f"""
--- VIDEO DESCRIPTION ---
{description}

--- TRANSCRIPT INTRO ---
{intro_text}

--- TRANSCRIPT OUTRO ---
{outro_text}
    """
    return corpus


In [16]:
def extract_dealer_details(contact_corpus):
    prompt = prompt_3_template.format(contact_corpus=contact_corpus)
    
    # Call the LLM for the first time
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-1106", # Using a more advanced model can yield better results
        response_format={"type": "json_object"}, 
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON. Your task is to extract dealer details from the provided text."},
            {"role": "user", "content": prompt}
        ]
    )
    response_content = response.choices[0].message.content
    try:
        # We can directly parse it without any cleaning
        if response_content:
            dealer_info = json.loads(response_content)
            return dealer_info
        else:
            print("No dealer information found in the response.")
            return None
    except json.JSONDecodeError:
        # This block is now mostly for safety; it should rarely be triggered
        print("Error: Model returned invalid JSON even in JSON mode.")
        return None

In [36]:
def extract_dealer_details_ollama(contact_corpus):
    prompt = prompt_3_template.format(contact_corpus=contact_corpus)
    
    # Call the LLM for the first time
    response_1 = ollama.chat(
        model="qwen3:30b", # Using a more advanced model can yield better results
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON. Your task is to extract dealer details from the provided text."},
            {"role": "user", "content": prompt}
        ], 
        think=True,
        format='json'
        # options={       # Optional: Adjust generation parameters if needed
        #     'temperature': 0.2,
        #     # 'num_predict': 1024 # Limit output length if necessary
        # }
    )
    response_content = response_1['message']['content']
    try:
        # We can directly parse it without any cleaning
        if response_content:
            dealer_info = json.loads(response_content)
            return dealer_info
        else:
            print("No dealer information found in the response.")
            return None
    except json.JSONDecodeError:
        # This block is now mostly for safety; it should rarely be triggered
        print("Error: Model returned invalid JSON even in JSON mode.")
        return None

In [34]:
temp_description = """Biggest Stock of New Shape Used Cars in Delhi, Top Quality Second Hand Cars in Delhi, Zoom Wheels

#secondhandcarsindelhincr 
#usedcar 
#preownedcar 
#delhicarmarket 


For Collaboration
Email Us on - ads.wowautos@gmail.com


Used cars in Delhi
Second Hand Cars In Delhi 
Cheapest Cars In Delhi 
Best used cars in delhi 
Budget used cars in delhi 
Pre owned cars in delhi 
Cheapest Second hand cars in delhi 
Second hand car in delhi 
Cheapest used cars in delhi 
Second hand in delhi for sale
Used car in delhi 
Used car market in delhi 
Used car dealership in delhi 
Delhi cheap cars 
Second hand car market in delhi 
Used cars 
Old cars in delhi
Secondhand car in delhi 
Used cars 
Cars 
Best used cars 
Good Cars 
Best used cars to buy 
Reliable cars 
Used cars to Buy 
Best Cars 
Reliable used cars 
Most reliable cars 
5 used cars 
Used cars you should buy 
Good Cars to buy 
Cars to buy 
Top 5 used cars to buy 
Best cars ever 
Top used cars 
Old cars delhi 
Old cars 
Rohini car market
Rohini cars

Less Driven Second Hand Cars in Delhi NCR,Top Trending Used Cars in Delhi NCR,Luxmi Cars,used cars in delhi,used cars in delhi cheap price,used cars in delhi latest videos,used cars in delhi under 3 lakh,used cars in delhi tamil,used cars in delhi swift diesel,second hand cars in delhi,second hand cars in delhi under 5 lakh,second hand cars in delhi under 10 lakhs,second hand cars in delhi latest video,second hand cars in delhi cheap price"""

In [37]:
contact_corpus = create_contact_corpus(structured_transcript, temp_description)
if contact_corpus:
    dealer_info = extract_dealer_details_ollama(contact_corpus)
    if dealer_info:
        df['dealer_name'] = dealer_info.get('dealer_name', 'Unknown')
        df['dealer_contact'] = dealer_info.get('dealer_contact', 'Unknown')
        df['dealer_email'] = dealer_info.get('dealer_email', 'Unknown')
        df['dealer_website'] = dealer_info.get('dealer_website', 'Unknown')
        df['dealer_location'] = dealer_info.get('dealer_location', 'Unknown')
    else:
        df[['dealer_name', 'dealer_contact', 'dealer_email', 'dealer_website', 'dealer_location']] = None

Error: Model returned invalid JSON even in JSON mode.


In [38]:
df

,OEM,Model,Variant,Price,Colour,Odo,Year,Service record,Frame type,Transmission type,...,video_id,video_title,published_at,channel_title,timestamp,dealer_name,dealer_contact,dealer_email,dealer_website,dealer_location
0,Tata,Safari,XZ A Plus,"₹1,99,000 (negotiable)",NA,NA,2021,NA,NA,Automatic,...,lbBDe4UJIgw,Some Title,2023-10-01 12:00:00,Some channel,2025-06-28 18:45:33,None,None,None,None,None
1,Tata,Punch,CNG,₹8.5 lakh (negotiable),NA,"5,700–7,000 km",2023,NA,NA,NA,...,lbBDe4UJIgw,Some Title,2023-10-01 12:00:00,Some channel,2025-06-28 18:45:33,None,None,None,None,None
2,Tata,Nexon,Fairless Plus,₹15 lakh,NA,NA,2023,NA,NA,Automatic,...,lbBDe4UJIgw,Some Title,2023-10-01 12:00:00,Some channel,2025-06-28 18:45:33,None,None,None,None,None
3,Tata,Nexon EV,Black Beauty,₹14.5 lakh,Black,"2,000 km",2024,NA,NA,NA,...,lbBDe4UJIgw,Some Title,2023-10-01 12:00:00,Some channel,2025-06-28 18:45:33,None,None,None,None,None
4,Tata,Nexon EV,LX Plus,₹25 lakh,NA,"25,000 km",2022,NA,NA,NA,...,lbBDe4UJIgw,Some Title,2023-10-01 12:00:00,Some channel,2025-06-28 18:45:33,None,None,None,None,None
5,Tata,Nexon EV,Prime,"₹8,85,000",NA,"60,000 km",2020,NA,NA,NA,...,lbBDe4UJIgw,Some Title,2023-10-01 12:00:00,Some channel,2025-06-28 18:45:33,None,None,None,None,None
6,Maruti,Altroz,Top,"₹6,99,000 (negotiable)",Dual Tone,NA,NA,NA,NA,Manual,...,lbBDe4UJIgw,Some Title,2023-10-01 12:00:00,Some channel,2025-06-28 18:45:33,None,None,None,None,None
7,Maruti,Altroz,Top,"₹7,45,000 (negotiable)",Dual Tone,NA,NA,NA,NA,Automatic,...,lbBDe4UJIgw,Some Title,2023-10-01 12:00:00,Some channel,2025-06-28 18:45:33,None,None,None,None,None
8,Maruti,Altroz,Top,"₹6,40,000 (negotiable)",White; Red beauty option,"5,200 km",2023,NA,NA,Automatic,...,lbBDe4UJIgw,Some Title,2023-10-01 12:00:00,Some channel,2025-06-28 18:45:33,None,None,None,None,None
9,Hyundai,i10,Sports,₹4.5 lakh,NA,"35,000 km",2019,NA,NA,Manual,...,lbBDe4UJIgw,Some Title,2023-10-01 12:00:00,Some channel,2025-06-28 18:45:33,None,None,None,None,None


In [44]:
def add_cars_to_db(cars):
    for car in cars:
        print(car.get('timestamp', None),
            car.get('video_id', None),
            car.get('published_at', None))
        break

In [45]:
add_cars_to_db(df.to_dict(orient='records'))

2025-06-28 18:45:33 lbBDe4UJIgw 2023-10-01 12:00:00
